In [19]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.optimize
import pandas.io.sql as psql
import pandas as pd
import sys
import xml.etree.ElementTree as ET

import requests
import re



In [20]:
%run ../../../lib/dwh.py

In [21]:
con=get_DWH()

In [278]:
%run indebtedness.py

In [4]:
%cd  C:\Users\Sean Violante\Documents\Projects\lendico\projects\returns\src

C:\Users\Sean Violante\Documents\Projects\lendico\projects\returns\src


# XML

In [46]:
scoring=psql.read_sql(""" select 
    * 
from backend.scoring_result sr
where sr.dwh_country_id=1 and sr.created_at>'2015-07-07' """,con)

In [260]:

z[0]

10762    [{"status":true,"actual":"retired","expected":...
10763    [{"status":true,"actual":"salaried","expected"...
10764    [{"status":true,"actual":"salaried","expected"...
10765    [{"status":true,"actual":"manual_worker","expe...
10766    [{"status":true,"actual":"retired","expected":...
Name: postcheck_data, dtype: object

In [261]:
import json

In [283]:
z.apply(json_extract,class_name='SchufaIndebtedness')

,actual,expected,operator,status
10762,6.45,17,<=,True
10763,9.26,15,<=,True
10764,11.26,14,<=,True
10765,4.87,18,<=,True
10766,8.94,20,<=,True
10767,31.16,20,<=,False
10768,4.06,14,<=,True
10769,113.76,16,<=,False
10770,NaN,NaN,None,None
10771,11.15,16,<=,True


In [308]:
scoring.is_latest.unique()

array([0, 1], dtype=int64)

In [286]:
z=pd.DataFrame(loan_requests.postcheck_data[-10:])

In [309]:
loan_requests=psql.read_sql("""
select 
    lr.id_loan_request, 
    lr.loan_request_nr, 
    sr.fk_user,
    sr.id,
    lr.principal_amount,
    ua.net_income,
    uh.postcheck_data
from base.loan_request lr
join base.user_account ua on 
    lr.dwh_country_id=ua.dwh_country_id and
    lr.fk_user=ua.id_user
join backend.loan_request_user_history uh on 
    lr.dwh_country_id=uh.dwh_country_id and
    lr.id_loan_request=uh.fk_loan_request
join backend.scoring_result sr on
    lr.dwh_country_id=sr.dwh_country_id and
    lr.fk_user=sr.fk_user
    
where sr.dwh_country_id=1 and sr.created_at>'2015-07-07'
and sr.is_latest=1
""",con)

In [310]:
indebt=loan_requests.postcheck_data.apply(json_extract,class_name='SchufaIndebtedness')


In [311]:
indebt.rename(columns=lambda x:'indebt:'+x, inplace=True)

In [298]:
z1.columns

Index([u'indebt:actual', u'indebt:expected', u'indebt:operator',
       u'indebt:status'],
      dtype='object')

In [312]:
loan_requests1=pd.concat((loan_requests,indebt), axis=1)

In [318]:
loan_requests1['indebt:actual'].notnull().sum()

135

In [50]:
scoring.columns

Index([    u'dwh_country_id',                 u'id',            u'fk_user',
                  u'request',           u'response',      u'provider_name',
                u'is_failed', u'ignore_for_scoring',              u'score',
       u'data_accessible_by',         u'created_at',         u'updated_at',
              u'dwh_created',  u'dwh_last_modified',       u'dwh_row_hash',
                u'is_latest'],
      dtype='object')

In [ ]:
scoring_ind=scoring.set_index(['dwh_country_id','id'])

In [236]:
scoring.score.head()

0      16
1    9999
2    9999
3    8536
4    9215
Name: score, dtype: object

In [48]:
scoring_ind["response_chk"]=scoring_ind["response"].str[:5]

In [49]:
scoring_ind["response"][(scoring_ind["response_chk"]=='s:17:')].head()

dwh_country_id  id   
1               99199    s:17:"ANONYMOUS_REQUEST";
                99230    s:17:"ANONYMOUS_REQUEST";
                99205    s:17:"ANONYMOUS_REQUEST";
                99206    s:17:"ANONYMOUS_REQUEST";
                99200    s:17:"ANONYMOUS_REQUEST";
Name: response, dtype: object

In [52]:
scoring_ind['response_x']=scoring_ind['response'].map(php_strip)
scoring_ind['response_len']=scoring_ind['response_x'].str.len()
scoring_ind['response_root']=scoring_ind['response_x'].map(xml_strip)
scoring_ind['response_merkmale']=scoring_ind['response_root'].map(schufa_parse)
merkmale=pd.concat(scoring_ind['response_merkmale'].values,keys=scoring_ind.index, names=['dwh_country_id','id','record'])
merkmale['Betrag']=merkmale['Betrag'].astype(np.float64) #is string, float to allow NAN
merkmale['Ratenzahl']=merkmale['Ratenzahl'].astype(np.float64)
merkmale['Date']=pd.to_datetime(merkmale.Datum,coerce=True, format='%d.%m.%Y') #pandas has problems with mixed data..need to select only real dates  

In [53]:
score_merkmale=scoring[['id_loan_request',u'loan_request_nr',  u'fk_user','id', 
                        'is_failed', u'ignore_for_scoring',u'score', 'created_at' ]].\
merge(merkmale.reset_index(), on='id')

KeyError: "['id_loan_request' u'loan_request_nr'] not in index"

In [18]:
score_merkmale.to_clipboard(encoding='UTF-8')

In [56]:
schufa_parse(xml_strip(scoring_ind['response_x'][3]))

,Beschreibung,Betrag,Betrag_waehrung,Datum,Merkmal_typ,Merkmalcode,Ratenart,Ratenzahl,Merkmal_id
0,Kredit,4669,EUR,23.08.2012,hauptmerkmal,KR,M,48,1
1,Erledigt,NaN,NaN,31.10.2012,erledigungsmerkmal,ER,NaN,NaN,1
2,Kredit,22228,EUR,09.11.2012,hauptmerkmal,KR,M,84,2
3,Erledigt,NaN,NaN,11.09.2014,erledigungsmerkmal,ER,NaN,NaN,2
4,Kredit,5364,EUR,01.09.2014,hauptmerkmal,KR,M,69,3
5,Kredit,27197,EUR,01.10.2014,hauptmerkmal,KR,M,84,4
6,Girokonto,NaN,NaN,NaN,hauptmerkmal,GI,NaN,NaN,5
7,Kreditkarte,NaN,NaN,12.08.2013,hauptmerkmal,CC,NaN,NaN,6
8,Kreditkarte mit Kreditrahmen,2300,EUR,15.04.2014,hauptmerkmal,CR,NaN,NaN,7


In [179]:
merkmale.columns

Index([u'Beschreibung', u'Betrag', u'Betrag_waehrung', u'Datum',
       u'Kontonummer', u'Merkmal_eigenesMerkmal', u'Merkmal_id',
       u'Merkmal_merkmalOhneGeburtsdatum', u'Merkmal_typ', u'Merkmalcode',
       u'Ratenart', u'Ratenzahl', u'not_creditworthy', u'not_creditworthy_US',
       u'not_creditworthy_KRMLKWKX_US', u'Date'],
      dtype='object')

#Not Creditworthy

In [87]:
merkmale['not_creditworthy']=merkmale.Merkmalcode.isin(['KW', 'RB', 'RV', 'S1', 'S2', 'S3', 'IE', 'EV', 'IA', 'HB', 'RA'])
KRML_US=match_id(merkmale, merkmale.Merkmalcode.isin(['KR','ML']))
KRML_US['not_creditworthy_US']=(KRML_US.Merkmalcode=='US')
KRMLKWKX_US=match_id(merkmale, merkmale.Merkmalcode.isin(['KR','ML','KW','KX']))
KRMLKWKX_US['not_creditworthy_US']=(KRMLKWKX_US.Merkmalcode=='US')
merkmale['not_creditworthy_US']=KRML_US['not_creditworthy_US']
merkmale['not_creditworthy_KRMLKWKX_US']=KRMLKWKX_US['not_creditworthy_US']


KR, ML with US

In [93]:
KRML_US['not_creditworthy_US'].sum()

14

In [92]:
KRMLKWKX_US['not_creditworthy_US'].sum()

201

In [88]:
RK=match_id(merkmale, merkmale.Merkmalcode.isin(['RK']))
RK.Merkmalcode.unique()

array(['XX', 'RK', 'ER'], dtype=object)

In [89]:
US_cases=match_id(merkmale, merkmale.Merkmalcode.isin(['US']))
US_cases.Merkmalcode.unique()

array(['KW', 'SD', 'US', 'SG', 'KR', 'KX', 'WS', 'SV', 'SE', 'ER'], dtype=object)

In [90]:
US_cases[US_cases.Merkmal_typ=='hauptmerkmal'].Merkmalcode.unique()

array(['KW', 'KR', 'KX'], dtype=object)

In [212]:
not_creditworthy_user=merkmale.groupby(level=['dwh_country_id','id'])['not_creditworthy',
                                                               'not_creditworthy_US',
                                                               'not_creditworthy_KRMLKWKX_US'].max()
# use max to find user with at least one row labelled not creditworthy
not_creditworthy_user.sum()

not_creditworthy                2219
not_creditworthy_US               12
not_creditworthy_KRMLKWKX_US     153
dtype: float64

In [96]:
merkmale.Merkmal_typ[merkmale.Merkmalcode=='RK'].unique()

array(['untermerkmal'], dtype=object)

In [431]:
132/3431

0.03847274846983387

In [417]:
merkmale['not_creditworthy_all']=merkmale['not_creditworthy'] | merkmale['not_creditworthy_US']

In [98]:
merkmale_user.not_creditworthy.mean()

0.56999743128692526

In [426]:
merkmale_user.not_creditworthy.shape

(3431L,)

In [315]:
merkmale.xs(99207,level='id')

Beschreibung  \
dwh_country_id record                                                      
1              0                                                  Kredit   
               1                                               Girokonto   
               2                                               Girokonto   
               3                                               Girokonto   
               4                                               Girokonto   
               5                                               Girokonto   
               6                                               Girokonto   
               7                                             Kreditkarte   
               8                                             Kreditkarte   
               9                                             Kreditkarte   
               10                                                     XX   
               11                                           Rahmenkredit   
               12                                                     XX   
               13                                           Rahmenkredit   
               14      Schuldner ist seiner Pflicht zur Abgabe der Ve...   
               15      Schuldner ist seiner Pflicht zur Abgabe der Ve...   

                      Betrag Betrag_waehrung        Datum        Kontonummer  \
dwh_country_id record                                                          
1              0         672             EUR   01.11.2012                NaN   
               1         NaN             NaN          NaN                NaN   
               2         NaN             NaN          NaN                NaN   
               3         NaN             NaN          NaN                NaN   
               4         NaN             NaN          NaN                NaN   
               5         NaN             NaN          NaN                NaN   
               6         NaN             NaN          NaN                NaN   
               7         NaN             NaN   31.01.2011                NaN   
               8         NaN             NaN   09.03.2010                NaN   
               9         NaN             NaN   19.02.2013                NaN   
               10        NaN             NaN          NaN                NaN   
               11        500             EUR  UNBEFRISTET                NaN   
               12        NaN             NaN          NaN                NaN   
               13        300             EUR  UNBEFRISTET                NaN   
               14        NaN             NaN   18.04.2013  D4501R00000213500   
               15        NaN             NaN   17.04.2013  D4501R00000253217   

                      Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id record                                      
1              0                         NaN           1   
               1                         NaN           2   
               2                         NaN           3   
               3                         NaN           4   
               4                         NaN           5   
               5                         NaN           6   
               6                         NaN           7   
               7                         NaN           8   
               8                         NaN           9   
               9                         NaN          10   
               10                        NaN          11   
               11                        NaN          11   
               12                        NaN          12   
               13                        NaN          12   
               14                        NaN          12   
               15                        NaN          12   

                      Merkmal_merkmalOhneGeburtsdatum    Merkmal_typ  \
dwh_country_id record                                                  
1              0         

## Additional Limit

In [223]:
additional_limit=match_id(merkmale, merkmale.Merkmalcode.isin(['XX','RK','CR']))
# add KG

additional_limit_paid_off=match_id(additional_limit, additional_limit.Merkmalcode=='ER')
additional_limit_outstanding=drop_paid_off(additional_limit)
additional_limit_outstanding_user=additional_limit_outstanding.groupby(level=['dwh_country_id','id']).Betrag.sum()
additional_limit_outstanding_user.name='add_Betrag'

In [85]:
additional_limit_outstanding_user.head()

dwh_country_id  id   
1               96444     3000
                96450     1000
                96452     1200
                96462    65000
                96463     7700
Name: Betrag, dtype: float64

In [83]:
additional_limit.xs(99207,level='id')

Beschreibung  Betrag Betrag_waehrung        Datum  \
dwh_country_id record                                                      
1              10                XX     NaN             NaN          NaN   
               11      Rahmenkredit     500             EUR  UNBEFRISTET   
               12                XX     NaN             NaN          NaN   
               13      Rahmenkredit     300             EUR  UNBEFRISTET   

                      Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id record                                                  
1              10             NaN                    NaN          11   
               11             NaN                    NaN          11   
               12             NaN                    NaN          12   
               13             NaN                    NaN          12   

                      Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id record                                                 
1              10                                 NaN  hauptmerkmal   
               11                                 NaN  untermerkmal   
               12                                 NaN  hauptmerkmal   
               13                                 NaN  untermerkmal   

                      Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id record                                                   
1              10              XX      NaN       NaN            False   
               11              RK      NaN       NaN            False   
               12              XX      NaN       NaN            False   
               13              RK      NaN       NaN            False   

                      not_creditworthy_US not_creditworthy_KRMLKWKX_US  
dwh_country_id record                                                   
1              10                     NaN                          NaN  
               11                     NaN                          NaN  
               12                     NaN                          NaN  
               13                     NaN                          NaN

In [386]:
additional_limit_outstanding_user.head()

dwh_country_id  id   
1               96444     3000
                96450     1000
                96452     1200
                96462    40000
                96463     7700
Name: Betrag, dtype: float64

In [373]:
print additional_limit.shape
print additional_limit_paid_off.shape
print additional_limit_outstanding.shape

(2585, 13)
(635, 13)
(1950, 14)


## Line of Credit

In [363]:
match_id(merkmale, merkmale.Merkmalcode.isin(['KG'])).Merkmalcode.unique()

array(['GI', 'KG', 'XX'], dtype=object)

In [224]:
LOC=match_id(merkmale, merkmale.Merkmalcode.isin(['GI']))
LOC=match_id(LOC, LOC.Merkmalcode=='KG')
LOC_paid_off=match_id(LOC, LOC.Merkmalcode=='ER')
LOC_outstanding=drop_paid_off(LOC)
LOC_outstanding_user=LOC_outstanding.groupby(level=['dwh_country_id','id']).Betrag.sum()
LOC_outstanding_user.name='LOC_Betrag'

In [115]:
additional_limit_outstanding_user.xs(96481,level='id')

dwh_country_id
1    2000
Name: Betrag, dtype: float64

In [111]:
LOC_outstanding_user[0:20]

dwh_country_id  id   
1               96445     3800
                96450      200
                96481     1000
                96482     1000
                96495      200
                96521      400
                96522     2500
                96526      700
                96553     6000
                96555     5000
                96556     6400
                96593     1200
                96596     1000
                96637     7525
                96655     4200
                96664    38000
                96674      100
                96692      500
                96712      500
                96726      500
Name: Betrag, dtype: float64

In [113]:
loan_requests[loan_requests.id==96481]

,id_loan_request,loan_request_nr,fk_user,id
391,118813,846475410,134354,96481


In [103]:
LOC_outstanding[LOC_outstanding.Merkmalcode=='KG']

Beschreibung  Betrag Betrag_waehrung Datum  \
dwh_country_id id    record                                              
1              99197 6         Girokonto     NaN             NaN   NaN   
               99210 1         Girokonto     NaN             NaN   NaN   
               99216 1         Girokonto     NaN             NaN   NaN   
                     2         Girokonto     NaN             NaN   NaN   
               99208 4         Girokonto     NaN             NaN   NaN   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 6              NaN                    NaN           5   
               99210 1              NaN                    NaN           2   
               99216 1              NaN                    NaN           2   
                     2              NaN                    NaN           3   
               99208 4              NaN                    NaN           4   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 6                                  NaN  hauptmerkmal   
               99210 1                                  NaN  hauptmerkmal   
               99216 1                                  NaN  hauptmerkmal   
                     2                                  NaN  hauptmerkmal   
               99208 4                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 6               GI      NaN       NaN            False   
               99210 1               GI      NaN       NaN            False   
               99216 1               GI      NaN       NaN            False   
                     2               GI      NaN       NaN            False   
               99208 4               GI      NaN       NaN            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99197 6                      NaN                          NaN   
               99210 1                      NaN                          NaN   
               99216 1                      NaN                          NaN   
                     2                      NaN                          NaN   
               99208 4                      NaN                          NaN   

                            paid  
dwh_country_id id    record       
1              99197 6       NaN  
               99210 1       NaN  
               99216 1       NaN  
                     2       NaN  
               99208 4       NaN

In [ ]:
# LOC can be time limited too?

In [101]:
LOC[LOC.Merkmalcode=='KG']

Beschreibung  Betrag Betrag_waehrung  \
dwh_country_id id     record                                                 
1              99219  11      Kredit zum Girokonto    9900             EUR   
               99278  1       Kredit zum Girokonto    6000             EUR   
               99315  7       Kredit zum Girokonto    5000             EUR   
               99313  6       Kredit zum Girokonto    1500             EUR   
               99333  2       Kredit zum Girokonto    1000             EUR   
               99354  1       Kredit zum Girokonto     475             EUR   
               99365  14      Kredit zum Girokonto    3400             EUR   
               99380  5       Kredit zum Girokonto     100             EUR   
               99392  12      Kredit zum Girokonto    3000             EUR   
                      14      Kredit zum Girokonto    3000             EUR   
               99412  8       Kredit zum Girokonto    1000             EUR   
               99414  4       Kredit zum Girokonto    2000             EUR   
               97137  19      Kredit zum Girokonto    3800             EUR   
               98002  2       Kredit zum Girokonto    1000             EUR   
               98139  2       Kredit zum Girokonto    1500             EUR   
               98239  25      Kredit zum Girokonto    2000             EUR   
               98242  2       Kredit zum Girokonto    3000             EUR   
               98705  3       Kredit zum Girokonto     500             EUR   
               98906  3       Kredit zum Girokonto    1000             EUR   
               98909  6       Kredit zum Girokonto    7700             EUR   
               99024  8       Kredit zum Girokonto    2000             EUR   
               99125  2       Kredit zum Girokonto    1500             EUR   
               96445  6       Kredit zum Girokonto    3800             EUR   
               96450  6       Kredit zum Girokonto     200             EUR   
               96481  5       Kredit zum Girokonto    1000             EUR   
               96482  7       Kredit zum Girokonto    1000             EUR   
               96495  2       Kredit zum Girokonto     200             EUR   
               96521  13      Kredit zum Girokonto     400             EUR   
               96522  5       Kredit zum Girokonto    2500             EUR   
               96526  1       Kredit zum Girokonto     700             EUR   
...                                            ...     ...             ...   
               100982 1       Kredit zum Girokonto    6400             EUR   
               100917 3       Kredit zum Girokonto    1000             EUR   
                      6       Kredit zum Girokonto     500             EUR   
               100957 3       Kredit zum Girokonto    1900             EUR   
               100967 6       Kredit zum Girokonto    1000             EUR   
               100954 4       Kredit zum Girokonto     500             EUR   
               100937 8       Kredit zum Girokonto   10000             EUR   
                      11      Kredit zum Girokonto    7000             EUR   
               101032 18      Kredit zum Girokonto     500             EUR   
               101043 2       Kredit zum Girokonto    7500             EUR   
               101001 13      Kredit zum Girokonto    5000             EUR   
               101090 3       Kredit zum Girokonto     500             EUR   
               101115 12      Kredit zum Girokonto     250             EUR   
               101117 12      Kredit zum Girokonto     250             EUR   
               101070 2       Kredit zum Girokonto    5500             EUR   
               101101 2       Kredit zum Girokonto     500             EUR   
               101062 9       Kredit zum Girokonto    2500             EUR   
               101008 6       Kredit zum Girokonto    2400             EUR   
               100925 12      Kredit zum Girokonto    1000  

In [105]:
LOC.shape

(4999, 15)

##Net Loans

In [182]:
net_loans=match_id(merkmale, merkmale.Merkmalcode.isin(['KR','ML']))
net_loans_exclude_US=drop_id(net_loans,net_loans.Merkmalcode=='US')
net_loans_exclude_US_outstanding=drop_paid_off(net_loans_exclude_US)


In [121]:
net_loans_exclude_US_outstanding.head()

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99197 4            Kredit    5364             EUR  01.09.2014   
                     5            Kredit   27197             EUR  01.10.2014   
               99210 0            Kredit    2299             EUR  15.05.2015   
               99216 0            Kredit    5000             EUR  15.11.2013   
               99208 2            Kredit    4869             EUR  01.06.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 4              NaN                    NaN           3   
                     5              NaN                    NaN           4   
               99210 0              NaN                    NaN           1   
               99216 0              NaN                    NaN           1   
               99208 2              NaN                    NaN           2   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 4                                  NaN  hauptmerkmal   
                     5                                  NaN  hauptmerkmal   
               99210 0                                  NaN  hauptmerkmal   
               99216 0                                  NaN  hauptmerkmal   
               99208 2                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 4               KR        M        69            False   
                     5               KR        M        84            False   
               99210 0               KR        M        36            False   
               99216 0               KR        M        60            False   
               99208 2               KR        M        36            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  
dwh_country_id id    record                                                   
1              99197 4                    False                        False  
                     5                    False                        False  
               99210 0                    False                        False  
               99216 0                    False                        False  
               99208 2                    False                        False

In [123]:
net_loans_exclude_US_outstanding.Beschreibung.unique()

array(['Kredit', 'Leasing', u'Saldo nach Gesamtf\xe4lligstellung',
       u'\xdcbergabe an ein Inkassounternehmen/eine interne Inkassostelle zur Beitreibung',
       'titulierter Saldo', 'Saldo', 'Forderungsverkauf',
       'uneinbringliche titulierte Forderung / Einzug unwirtschaftlich',
       'Widerspruch zum Saldo',
       u'R\xfcckgabe vom Inkassounternehmen/von einer internen Inkassostelle an den Gl\xe4ubiger'], dtype=object)

In [159]:
def calc_installments(issued,now):
    return (now.year-issued.year)*12 + (now.month-issued.month) + 1 - 1*(issued.day>now.day)

In [166]:
12+7-9+1

11

In [245]:
a=net_loans_exclude_US_outstanding[net_loans_exclude_US_outstanding.Ratenart=='J']
a.to_clipboard()

In [169]:
net_loans_exclude_US_outstanding.Ratenart.unique()

array(['M', nan, 'J'], dtype=object)

In [185]:
91//12

7

In [191]:
net_loans_exclude_US_outstanding.loc[KR_select & (net_loans_exclude_US_outstanding.Ratenart=='J')]

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99315 0            Kredit   60000             EUR  30.12.2007   
               96453 0            Kredit    7414             EUR  31.03.2001   
               96780 2            Kredit   20250             EUR  28.02.2010   
               98778 0            Kredit    9900             EUR  31.05.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99315 0              NaN                    NaN           1   
               96453 0              NaN                    NaN           1   
               96780 2              NaN                    NaN           3   
               98778 0              NaN                    NaN           1   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99315 0                                  NaN  hauptmerkmal   
               96453 0                                  NaN  hauptmerkmal   
               96780 2                                  NaN  hauptmerkmal   
               98778 0                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart  Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                    
1              99315 0               KR        J          8            False   
               96453 0               KR        J         15            False   
               96780 2               KR        J          8            False   
               98778 0               KR        J         10            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99315 0                    False                        False   
               96453 0                    False                        False   
               96780 2                    False                        False   
               98778 0                    False                        False   

                                  Date  paid_installments  installment  
dwh_country_id id    record                                             
1              99315 0      2007-12-30                  7         9750  
               96453 0      2001-03-31                 14          642  
               96780 2      2010-02-28                  5         3290  
               98778 0      2014-05-31                  1         1287

In [214]:
now_date=pd.Timestamp('2015-07-17')
KR_select=net_loans_exclude_US_outstanding.Merkmalcode=='KR'
net_loans_exclude_US_outstanding.loc[KR_select,'paid_installments'] = \
    calc_installments(net_loans_exclude_US_outstanding.loc[KR_select,'Date'].dt,now_date)
    
net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl']
net_loans_exclude_US_outstanding.loc[KR_select & (net_loans_exclude_US_outstanding.Ratenart=='J'),'Ratenzahl_Month'] = \
    12 *net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl']

net_loans_exclude_US_outstanding.loc[KR_select,'paid_installments'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,['Ratenzahl_Month','paid_installments']].min(axis=1)
           
net_loans_exclude_US_outstanding.loc[KR_select,'installment'] = \
    (1.3*net_loans_exclude_US_outstanding.loc[KR_select,'Betrag']).\
    floordiv(net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month'])
    
net_loans_exclude_US_outstanding.loc[KR_select,'loan_gross_amount'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,'installment'] * \
    net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month']

net_loans_exclude_US_outstanding.loc[KR_select,'loan_gross_outstanding_amount'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,'loan_gross_amount'] - \
    net_loans_exclude_US_outstanding.loc[KR_select,'installment'] * \
    net_loans_exclude_US_outstanding.loc[KR_select,'paid_installments']

net_loans_exclude_US_outstanding.loc[KR_select,'outstanding_installments'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month'] - \
    net_loans_exclude_US_outstanding.loc[KR_select,'paid_installments']

net_loans_exclude_US_outstanding.loc[KR_select,'interest'] = \
    net_loans_exclude_US_outstanding.loc[KR_select,'loan_gross_amount'] - \
    net_loans_exclude_US_outstanding.loc[KR_select,'Betrag']
    
net_loans_exclude_US_outstanding.loc[KR_select,'interest_allowance'] = \
    (net_loans_exclude_US_outstanding.loc[KR_select,'interest'].mul(
    net_loans_exclude_US_outstanding.loc[KR_select,'outstanding_installments']-4).mul(
    net_loans_exclude_US_outstanding.loc[KR_select,'outstanding_installments']-3)/ 
    (net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month'].mul(
        net_loans_exclude_US_outstanding.loc[KR_select,'Ratenzahl_Month']+1))).round(2)
net_loans_exclude_US_outstanding.loc[KR_select,'net_value'] = np.maximum( 0,
    net_loans_exclude_US_outstanding.loc[KR_select,'loan_gross_outstanding_amount'] - \
    net_loans_exclude_US_outstanding.loc[KR_select,'interest_allowance'])


In [198]:
lr=118813
loan_requests[loan_requests.id_loan_request==lr]

,id_loan_request,loan_request_nr,fk_user,id
391,118813,846475410,134354,96481


In [211]:
merkmale.head().to_clipboard()

In [206]:
id=99315
loan_requests[loan_requests.id==id]

,id_loan_request,loan_request_nr,fk_user,id
62,122080,767115004,137604,99315


In [225]:
net_loans_exclude_US_outstanding_user=net_loans_exclude_US_outstanding.groupby(level=['dwh_country_id','id'])\
['Betrag','loan_gross_amount','loan_gross_outstanding_amount', 'interest','interest_allowance', 'net_value'] .sum()
net_loans_exclude_US_outstanding_user.rename(columns={'Betrag':'net_loan_Betrag'},inplace=True)

In [227]:
all_user=pd.concat((not_creditworthy_user, 
                   additional_limit_outstanding_user,  
                   LOC_outstanding_user, 
                   net_loans_exclude_US_outstanding_user),axis=1)

In [239]:
merkmale.xs(96439,level='id')

Beschreibung  Betrag  \
dwh_country_id record                                            
1              0              Kredit mit Endfälligkeit     100   
               1                              Erledigt     NaN   
               2              Kredit mit Endfälligkeit     100   
               3                              Erledigt     NaN   
               4              Kredit mit Endfälligkeit     100   
               5                              Erledigt     NaN   
               6              Kredit mit Endfälligkeit     200   
               7                              Erledigt     NaN   
               8              Kredit mit Endfälligkeit     200   
               9                              Erledigt     NaN   
               10             Kredit mit Endfälligkeit     200   
               11                             Erledigt     NaN   
               12             Kredit mit Endfälligkeit     200   
               13      Saldo nach Gesamtfälligstellung     300   
               14                            Girokonto     NaN   

                      Betrag_waehrung        Datum Kontonummer  \
dwh_country_id record                                            
1              0                  EUR  UNBEFRISTET         NaN   
               1                  NaN   05.08.2014         NaN   
               2                  EUR  UNBEFRISTET         NaN   
               3                  NaN   05.09.2014         NaN   
               4                  EUR  UNBEFRISTET         NaN   
               5                  NaN   02.10.2014         NaN   
               6                  EUR  UNBEFRISTET         NaN   
               7                  NaN   03.11.2014         NaN   
               8                  EUR  UNBEFRISTET         NaN   
               9                  NaN   04.12.2014         NaN   
               10                 EUR  UNBEFRISTET         NaN   
               11                 NaN   06.01.2015         NaN   
               12                 EUR  UNBEFRISTET         NaN   
               13                 EUR   26.03.2015         NaN   
               14                 NaN          NaN         NaN   

                      Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id record                                      
1              0                         NaN           1   
               1                         NaN           1   
               2                         NaN           2   
               3                         NaN           2   
               4                         NaN           3   
               5                         NaN           3   
               6                         NaN           4   
               7                         NaN           4   
               8                         NaN           5   
               9                         NaN           5   
               10                        NaN           6   
               11                        NaN           6   
               12                        NaN           7   
               13                        NaN           7   
               14                        NaN           8   

                      Merkmal_merkmalOhneGeburtsdatum         Merkmal_typ  \
dwh_country_id record                                                       
1              0                                  NaN        hauptmerkmal   
               1                                  NaN  erledigungsmerkmal   
               2                                  NaN        hauptmerkmal   
               3                                  NaN  erledigungsmerkmal   
               4                                  NaN        hauptmerkmal   
               5                                  NaN  erledigungsmerkmal   
               6                                  NaN        hauptmerkmal   
               7                                  NaN  erledigungsmerkmal   
     

In [240]:
loan_requests[loan_requests.id==96439]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount
348,118766,484010163,134308,96439,3000


In [228]:
all_user.head()

not_creditworthy not_creditworthy_US  \
dwh_country_id id                                           
1              96439            False                 NaN   
               96440             True                 NaN   
               96441             True                 NaN   
               96442            False               False   
               96443             True               False   

                     not_creditworthy_KRMLKWKX_US  add_Betrag  LOC_Betrag  \
dwh_country_id id                                                           
1              96439                        False         NaN         NaN   
               96440                        False         NaN         NaN   
               96441                        False         NaN         NaN   
               96442                        False         NaN         NaN   
               96443                        False         NaN         NaN   

                      net_loan_Betrag  loan_gross_amount  \
dwh_country_id id                                          
1              96439              NaN                NaN   
               96440              NaN                NaN   
               96441              NaN                NaN   
               96442            33975              44016   
               96443             8844                NaN   

                      loan_gross_outstanding_amount  interest  \
dwh_country_id id                                               
1              96439                            NaN       NaN   
               96440                            NaN       NaN   
               96441                            NaN       NaN   
               96442                          35894     10041   
               96443                            NaN       NaN   

                      interest_allowance  net_value  
dwh_country_id id                                    
1              96439                 NaN        NaN  
               96440                 NaN        NaN  
               96441                 NaN        NaN  
               96442             6302.86   29591.14  
               96443                 NaN        NaN

In [313]:
lr_all_user=loan_requests1.merge(all_user.reset_index(),on='id')

In [314]:
lr_all_user['net_debt']=lr_all_user[['net_value','principal_amount','LOC_Betrag','add_Betrag']].sum(axis=1) # sum removes NAN
lr_all_user['debt_ratio']=lr_all_user['net_debt']/lr_all_user['net_income']

In [315]:
lr_all_user[lr_all_user['indebt:actual'].notnull()].to_clipboard()

In [316]:
loan_requests1.shape

(3340, 11)

In [317]:
(loan_requests.postcheck_data!='null').sum()

1644

In [244]:
lr_all_user[lr_all_user.id_loan_request==120973].T.to_clipboard()

In [217]:
net_loans_exclude_US_outstanding_user.head()

Betrag  loan_gross_amount  \
dwh_country_id id                                 
1              96442   33975              44016   
               96443    8844                NaN   
               96444     183                230   
               96445   28749              21720   
               96446   20000              25956   

                      loan_gross_outstanding_amount  interest  \
dwh_country_id id                                               
1              96442                          35894     10041   
               96443                            NaN       NaN   
               96444                              0        47   
               96445                           7610      4943   
               96446                          24411      5956   

                      interest_allowance  net_value  
dwh_country_id id                                    
1              96442             6302.86   29591.14  
               96443                 NaN        NaN  
               96444                5.13       0.00  
               96445              492.99    7117.01  
               96446             4754.79   19656.21

In [178]:
net_loans_exclude_US_outstanding.loc[KR_select].head()

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99197 4            Kredit    5364             EUR  01.09.2014   
                     5            Kredit   27197             EUR  01.10.2014   
               99210 0            Kredit    2299             EUR  15.05.2015   
               99216 0            Kredit    5000             EUR  15.11.2013   
               99208 2            Kredit    4869             EUR  01.06.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 4              NaN                    NaN           3   
                     5              NaN                    NaN           4   
               99210 0              NaN                    NaN           1   
               99216 0              NaN                    NaN           1   
               99208 2              NaN                    NaN           2   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 4                                  NaN  hauptmerkmal   
                     5                                  NaN  hauptmerkmal   
               99210 0                                  NaN  hauptmerkmal   
               99216 0                                  NaN  hauptmerkmal   
               99208 2                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 4               KR        M        69            False   
                     5               KR        M        84            False   
               99210 0               KR        M        36            False   
               99216 0               KR        M        60            False   
               99208 2               KR        M        36            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99197 4                    False                        False   
                     5                    False                        False   
               99210 0                    False                        False   
               99216 0                    False                        False   
               99208 2                    False                        False   

                                  Date  paid_installments  
dwh_country_id id    record                                
1              99197 4      2014-09-01                 11  
                     5      2014-10-01                 10  
               99210 0      2015-05-15                  3  
               99216 0      2013-11-15                 21  
               99208 2      2014-06-01                 14

In [306]:
loan_requests[loan_requests.id_loan_request==122466]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data
6687,122466,44200180,7001,99634,1500,2198,"[{""status"":true,""actual"":""retired"",""expected"":..."
6693,122466,44200180,7001,99634,1500,2198,null


In [125]:
z1=match_id(net_loans_exclude_US_outstanding,net_loans_exclude_US_outstanding.Beschreibung==u'R\xfcckgabe vom Inkassounternehmen/von einer internen Inkassostelle an den Gl\xe4ubiger')
z1

Beschreibung  \
dwh_country_id id     record                                                      
1              100654 3                                                  Kredit   
                      4                         Saldo nach Gesamtfälligstellung   
                      5                                                   Saldo   
                      6                                                   Saldo   
                      7                                                   Saldo   
                      8                                                   Saldo   
                      9                                                   Saldo   
                      10                                                  Saldo   
                      11                                                  Saldo   
                      12                                                  Saldo   
                      13                                                  Saldo   
                      14                                                  Saldo   
                      15                                                  Saldo   
                      16                                                  Saldo   
                      17                                                  Saldo   
                      18                                                  Saldo   
                      19                                                  Saldo   
                      20                                                  Saldo   
                      21                                                  Saldo   
                      22                                                  Saldo   
                      23                                                  Saldo   
                      24                                                  Saldo   
                      25                                                  Saldo   
                      26                                                  Saldo   
                      27                                                  Saldo   
                      28                                                  Saldo   
                      29                                                  Saldo   
                      30                                                  Saldo   
                      31                                                  Saldo   
                      32                                                  Saldo   
                      33                                                  Saldo   
                      34                                                  Saldo   
                      35                                                  Saldo   
                      36                                                  Saldo   
                      37                                                  Saldo   
                      38                                                  Saldo   
                      39                                                  Saldo   
                      40      Rückgabe vom Inkassounternehmen/von einer inte...   

                              Betrag Betrag_waehrung       Datum Kontonummer  \
dwh_country_id id     record                                                   
1              100654 3        35023             EUR  15.04.2010         NaN   
                      4        17388             EUR  24.05.2012         NaN   
                      5        17402             EUR  31.05.2012         NaN   
                      6        17432             EUR  13.06.2012         NaN   
                      7        17458             EUR  16.08.2012         NaN   
                      8        17518             EUR  12.09.2012         NaN   
                      9        17587             EUR  12.10.2012         NaN   
                      10  

In [ ]:
net_loans_exclude_US_outstanding_user.xs()

## Investigation

In [ ]:
merkmale[merkmale.Merkmal_merkmalOhneGeburtsdatum=='ja'].to_clipboard(encoding='UTF-8')

In [ ]:
identify which 

In [348]:
d.head()

,dwh_country_id,id,record,Beschreibung,Betrag,Betrag_waehrung,Datum,Kontonummer,Merkmal_eigenesMerkmal,Merkmal_id,Merkmal_merkmalOhneGeburtsdatum,Merkmal_typ,Merkmalcode,Ratenart,Ratenzahl,not_creditworthy
0,1,99207,10,XX,NaN,NaN,NaN,NaN,NaN,11,NaN,hauptmerkmal,XX,NaN,NaN,False
1,1,99207,11,Rahmenkredit,500,EUR,UNBEFRISTET,NaN,NaN,11,NaN,untermerkmal,RK,NaN,NaN,False
2,1,99207,12,XX,NaN,NaN,NaN,NaN,NaN,12,NaN,hauptmerkmal,XX,NaN,NaN,False
3,1,99207,13,Rahmenkredit,300,EUR,UNBEFRISTET,NaN,NaN,12,NaN,untermerkmal,RK,NaN,NaN,False
4,1,99219,12,XX,NaN,NaN,NaN,NaN,NaN,10,NaN,hauptmerkmal,XX,NaN,NaN,False


In [397]:
map(lambda x:pd.Series(x[1].unique(),name=x[0]),merkmale[
        [u'Beschreibung',  u'Betrag_waehrung','Datum', 
         'Merkmal_eigenesMerkmal','Merkmal_merkmalOhneGeburtsdatum',
         'Merkmal_typ',
         u'Merkmalcode',
       u'Ratenart', u'Ratenzahl' ]].iteritems())

[0                                      Abwicklungskonto
 1                                                 Saldo
 2                                     titulierter Saldo
 3     Gläubigerbefriedigung ausgeschlossen/nicht nac...
 4                                                Kredit
 5                                              Erledigt
 6                                             Girokonto
 7                                           Kreditkarte
 8                          Kreditkarte mit Kreditrahmen
 9                                    Anfrage zum Kredit
 10    Schuldner ist seiner Pflicht zur Abgabe der Ve...
 11               grundpfandrechtlich gesicherter Kredit
 12                               Anfrage zur Bürgschaft
 13                                              Leasing
 14    Investitionskredit (für Selbständige und Klein...
 15                                                   XX
 16                                         Rahmenkredit
 17                          Mi

In [293]:
merkmale.Merkmal_typ.unique()

array(['hauptmerkmal', 'untermerkmal', 'einzelmerkmal',
       'erledigungsmerkmal'], dtype=object)

In [219]:
scoring_ind['response_x'][(scoring_ind['response_root'].isnull())].unique()

array([u'ANONYMOUS_REQUEST', u'deleted_user'], dtype=object)

In [258]:
merkmale.to_clipboard()

In [214]:
scoring_ind.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4305 entries, (1, 99213) to (1, 100774)
Data columns (total 18 columns):
fk_user               4305 non-null int64
request               4305 non-null object
response              4305 non-null object
provider_name         4305 non-null object
is_failed             4305 non-null int64
ignore_for_scoring    4305 non-null int64
score                 4305 non-null object
data_accessible_by    4305 non-null float64
created_at            4305 non-null datetime64[ns]
updated_at            4305 non-null datetime64[ns]
dwh_created           4305 non-null datetime64[ns]
dwh_last_modified     4305 non-null datetime64[ns]
dwh_row_hash          4305 non-null object
is_latest             4305 non-null int64
response_chk          4305 non-null object
response_x            4305 non-null object
response_len          4305 non-null int64
response_root         3707 non-null object
dtypes: datetime64[ns](4), float64(1), int64(5), object(8)
memory usage: 65

In [205]:
merkmale.iloc[3]

,Beschreibung,Betrag,Betrag_waehrung,Datum,Merkmal_typ,Merkmalcode,Merkmal_id
0,Girokonto,NaN,NaN,NaN,hauptmerkmal,GI,1
1,Abwicklungskonto,NaN,NaN,06.05.2013,hauptmerkmal,KW,2
2,Saldo,135,EUR,06.05.2013,untermerkmal,SD,2
3,Saldo,153,EUR,28.10.2013,untermerkmal,SD,2


# Web Requests

In [13]:
payload={'username':'sean.violante@lendico.de',
         'password': '7642lottA'}

with requests.Session() as s:
    p = s.post('https://admin.lendico.de/admin/', data=payload, auth=('lendico_beta', 'p2pLend4every1'))
    # print the html returned or something more intelligent to see if it's a successful login page.
    print p.text
    print "HELLO"
    # An authorised request.
    r = s.get('https://admin.lendico.de/admin/loan-request/detail/123290',auth=('lendico_beta', 'p2pLend4every1'))
    print r.text

<!DOCTYPE html>
<html>
<head><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o?o:n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({QJf3ax:[function(e,n){function t(e){function n(n,t,a){e&&e(n,t,a),a||(a={});for(var u=c(n),f=u.length,s=i(a,o,r),p=0;f>p;p++)u[p].apply(s,t);return s}function a(e,n){f[e]=c(e).concat(n)}function c(e){return f[e]||[]}function u(){return t(n)}var f={};return{on:a,emit:n,create:u,listeners:c,_events:f}}function r(){return{}}var o="nr@context",i=e("gos");n.exports=t()},{gos:"7eSDFh"}],ee:[function(e,n){n.exports=e("QJf3ax")},{}],3:[function(e,n){function t(e){return function(){r(e,[(new Date).getTime()].concat(i(arguments)))}}var r=e("handle"),o=e(1),i=e(2);"undefined"==typeof window.newrelic&&(newrelic=window.NREUM);var a=["setPag